# Implementación 
### Equipo: 

In [1]:
# Parámetros generales del notebook
ruta_bases = 'bases/'
sample_size = 200000
cv = 3

In [2]:
# Preparación ambiente AWS SageMaker
#!pip install --upgrade pip
#!pip install xgboost
#!pip install lightgbm
#!pip install -U scikit_learn
#!pip install -U pandas

In [3]:
# Importación de librerías
import json
import glob
import pandas as pd
import numpy as np
#import missingno as msgno
import matplotlib.pyplot as plt
import funciones as fn
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from ml_classes import PrepML, MLModel
from matplotlib.pyplot import rcParams
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from lib.get_nhtsa_json import get_nhtsa_json

In [4]:
# Parámetros generales para plots
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 15, 8
# Semilla pseudo-aleatoria
rd_seed = 1234

# 0. Obtener la información

Esta es una demostración del proceso realizado por `get_features.py` para poder obtener información relacionada con el `Vin` de los vehículos a través de una API. El proceso consite en los siguientes pasos:
* 1.- Extraer todos los `Vin` de la base completa `true_car_listings.csv`.
* 2.- Requerir a través de la Api 'chunks' de 50 registros por cada petición.
* 3.- Guardar en la memoria el json en formato texto, agregando 50 registros por cada iteración.
* 4.- Una vez completada las iteraciones guardar en formato json todos los registros requeridos.

In [5]:
# Requerimos todos los 'Vin'    
all_vins = pd.read_csv(f'{ruta_bases}true_car_listings.csv')['Vin'].to_list()
# Parámetros de muestra
start = 8
end = 8

json_text = '['
for i in range(start, end+1):
    # Generar requerimiento con 50 registros Vin
    vin_list = all_vins[50 * (i - 1):50 * i]
    json_text += get_nhtsa_json(vin_list, i)

# Cerrar lista de Json
json_text = json_text[:-2] + ']'
# Exportar resultados a archivo json
with open(f'api_test/data_{start}_{end}.json', 'w') as json_file:
    json_file.write(json_text)

8: 11.0s


Paralelamente, en base a una muestra de la base total, se definió el primer filtro de variables requeridas a través la API: que tengan menos del 10% de datos perdidos, las cuales se presentan a continaución:

In [6]:
cols = ['AirBagLocFront', 'BodyClass', 'BusFloorConfigType', 'BusType',
       'CustomMotorcycleType', 'DisplacementCC', 'DisplacementCI',
       'DisplacementL', 'Doors', 'EngineCylinders', 'EngineHP', 'EngineKW',
       'ErrorCode', 'ErrorText', 'FuelTypePrimary', 'Make', 'Manufacturer',
       'ManufacturerId', 'Model', 'ModelYear', 'MotorcycleChassisType',
       'MotorcycleSuspensionType', 'PlantCity', 'PlantCountry', 'TPMS',
       'TrailerBodyType', 'TrailerType', 'VIN', 'VehicleType']

Con estas columnas seleccionadas, se procede a importar los archivos json (varios en el proceso original) para luego mapearlos para retraer solo aquellas columnas, creando un DataFrame con ellas y luego exportarlas en un csv.

In [7]:
# Importación de archivo json
filenames = glob.glob('api_test/*.json')
json_list = []

for filename in filenames:
    print(filename)
    with open(filename, 'r') as file:
        # Mapeamos considerando solo las columnas seleccionadas
        data = list(map(fn.get_info, 
                        json.loads(file.read())
                       )
                   )
    json_list += data

# Creación y exortación de DataFrame con features extraídos
data_json = pd.DataFrame(data=json_list,
                         columns=cols)
data_json.to_csv('api_test/data_api.csv')

api_test/data_8_8.json


ValueError: 29 columns passed, passed data had 0 columns

# 1. Creación del Dataset 

## 1.1 Bases Originales

In [8]:
# Importación de las bases para muestras de entrenamiento y prueba
df_train = pd.read_csv(f'{ruta_bases}true_cars_train.csv',
                       delimiter=";")
df_test = pd.read_csv(f'{ruta_bases}true_cars_test.csv',
                      delimiter=";")
# Dimensiones de las bases
print(f'Base Train: {df_train.shape}\nBase Test: {df_test.shape}')

Base Train: (639145, 8)
Base Test: (212977, 8)


In [9]:
# Información general de muestra de entrenamiento
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639145 entries, 0 to 639144
Data columns (total 8 columns):
Price      639145 non-null int64
Year       639145 non-null int64
Mileage    639145 non-null int64
City       639145 non-null object
State      639145 non-null object
Vin        639145 non-null object
Make       639145 non-null object
Model      639145 non-null object
dtypes: int64(3), object(5)
memory usage: 39.0+ MB


In [10]:
# Creación de atributo 'sample'
df_train['sample'] = 'train'
df_test['sample'] = 'test'

In [11]:
# Unión de ambas bases
df_data = pd.concat([df_train, df_test])
print(f'Base Data: {df_data.shape}')

Base Data: (852122, 9)


## 1.2 Base API

In [12]:
# Importación de la base extraída por el requerimiento a la api
df_api = pd.read_csv(f'{ruta_bases}api_features.csv').drop(columns='Unnamed: 0')
print(f'Base API: {df_api.shape}')

Base API: (846562, 29)


In [13]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846562 entries, 0 to 846561
Data columns (total 29 columns):
AirBagLocFront              797265 non-null object
BodyClass                   845535 non-null object
BusFloorConfigType          843524 non-null object
BusType                     843524 non-null object
CustomMotorcycleType        846525 non-null object
DisplacementCC              839096 non-null float64
DisplacementCI              839096 non-null float64
DisplacementL               839096 non-null float64
Doors                       739918 non-null float64
EngineCylinders             735744 non-null object
EngineHP                    380764 non-null object
EngineKW                    380764 non-null object
ErrorCode                   846550 non-null object
ErrorText                   846550 non-null object
FuelTypePrimary             789859 non-null object
Make                        846525 non-null object
Manufacturer                846525 non-null object
ManufacturerId    

In [14]:
# Identificar columnas con solo valores "Not Applicable" 
notapp_series = df_api\
                    .isin(['Not Applicable'])\
                    .sum()
cols2drop = list(notapp_series[notapp_series > 1].index)

In [15]:
# Identificar columnas con más de un 15% de datos perdidos
null_series = df_api\
                .isnull()\
                .sum()\
                /df_api.shape[0] 
cols2drop += list(null_series[null_series > .15].index)

In [16]:
# Columnas repetidas en la data original
cols2drop += ['ModelYear', 'Make']

In [17]:
# Borrar aquellas columnas 
df_api = df_api.drop(columns=cols2drop)
print(f'Base API: {df_api.shape}')

Base API: (846562, 16)


In [18]:
df_api=df_api.rename(columns={"AirBagLocFront":"Airag_LocFront",
                        "BodyClass":"d_Body_Class",
                        "DisplacementCC":"d_DisplacementCC",
                        "DisplacementCI":"d_DisplacementCI",
                        "DisplacementL":"d_DisplacementL",
                        "Doors":"d_Doors",
                        "EngineCylinders ":"d_EngineCylinders",
                        "EngineHP":"d_EngineHP",
                        "EngineKW":"d_EngineKW",
                        "ErrorCode":"d_ErrorCode",
                        "ErrorText":"d_ErrorText",
                        "FuelTypePrimary":"d_FuelTypePrimary",
                        "Make":"d_Make",
                        "Manufacturer":"d_Manufacturer",
                        "ManufacturerId":"d_ManufacturerId",
                        "Model":"d_Model",
                        "ModelYear":"d_ModelYear",
                        "PlantCity":"d_PlantCity",
                        "PlantCountry":"d_PlantCountry",
                        "TPMS":"d_TPMS",
                        "VIN":"Vin",
                        "VehicleType":"d_VehicleType"})

## 1.3 Unión de Bases

In [19]:
# Unión de bases
df = pd.merge(left=df_data, 
              right=df_api, 
              how='inner',
              on='Vin')
# Dimensiones de la base
print(f'Dataset: {df.shape}')

Dataset: (846644, 24)


In [20]:
# Información general de la base
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 846644 entries, 0 to 846643
Data columns (total 24 columns):
Price                846644 non-null int64
Year                 846644 non-null int64
Mileage              846644 non-null int64
City                 846644 non-null object
State                846644 non-null object
Vin                  846644 non-null object
Make                 846644 non-null object
Model                846644 non-null object
sample               846644 non-null object
Airag_LocFront       797359 non-null object
d_Body_Class         845629 non-null object
d_DisplacementCC     839190 non-null float64
d_DisplacementCI     839190 non-null float64
d_DisplacementL      839190 non-null float64
d_Doors              740010 non-null float64
EngineCylinders      735838 non-null object
d_ErrorCode          846644 non-null object
d_ErrorText          846644 non-null object
d_FuelTypePrimary    789951 non-null object
d_Manufacturer       846619 non-null object
d_Manufa

In [21]:
# Filtrar 30 modelos
modelos = df['Make'].value_counts()
df = df[df['Make'].isin(modelos[modelos > 30].index)]

In [22]:
clase = df['d_Body_Class'].value_counts()
df = df[df['d_Body_Class'].isin(clase[clase > 1000].index)]

# 2. Análisis exploratorio de datos

## 2.1 Análisis de los datos perdidos

In [23]:
#msgno.matrix(df)

# 3. Preproceso

In [24]:
# Liberar Espacio Memoria
del df_api
del df_data
del df_train
del df_test

In [25]:
# Selección de variables para modelos
select_vars = ['Price', 'Mileage', 'sample', 'd_Body_Class', 'Make']
#sample_size = df[select_vars].shape[0]

In [26]:
# Muestra aleatoria
df_sample = df[select_vars]\
                .sample(sample_size)\
                .reset_index(drop=True)\

In [27]:
# Instanciar clase para realizar preproceso
df_prep = PrepML(df_sample)

In [28]:
# Realizamos OneHot Encoder a las columnas categóricas seleccionadas
df_prep.one_hot_encoder(['d_Body_Class', 'Make'])
df_prep.df

,Price,Mileage,sample,d_Body_Class_Sport_Utility_Vehicle__SUV__Multi_Purpose_Vehicle__MPV_,d_Body_Class_Wagon,d_Body_Class_Pickup,d_Body_Class_Hatchback_Liftback_Notchback,d_Body_Class_Coupe,d_Body_Class_Minivan,d_Body_Class_Convertible_Cabriolet,...,Make_Genesis,Make_Oldsmobile,Make_Aston,Make_Lamborghini,Make_Isuzu,Make_Rolls_Royce,Make_Alfa,Make_Plymouth,Make_McLaren,Make_Lotus
0,13699,33452,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14920,73732,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29990,65322,test,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15000,61629,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14805,101321,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,18951,39260,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199996,2500,119832,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199997,13000,32178,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199998,24488,49827,test,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# Removemos outliers (opcional)
#df_prep.remove_outliers(['Price', 'Mileage'], multiplier=1.5)
#df_prep.df

In [29]:
# Estandarizamos variables continuas seleccionadas
df_prep.standard_scaler(['Mileage'])
df_prep.df

,Price,sample,d_Body_Class_Sport_Utility_Vehicle__SUV__Multi_Purpose_Vehicle__MPV_,d_Body_Class_Wagon,d_Body_Class_Pickup,d_Body_Class_Hatchback_Liftback_Notchback,d_Body_Class_Coupe,d_Body_Class_Minivan,d_Body_Class_Convertible_Cabriolet,d_Body_Class_Van,...,Make_Oldsmobile,Make_Aston,Make_Lamborghini,Make_Isuzu,Make_Rolls_Royce,Make_Alfa,Make_Plymouth,Make_McLaren,Make_Lotus,Mileage
0,13699,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.455180
1,14920,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.522134
2,29990,test,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.318082
3,15000,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.228479
4,14805,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.191527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,18951,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.314261
199996,2500,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.640660
199997,13000,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.486091
199998,24488,test,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.057873


In [30]:
# Separar muestras según
X_train, y_train, X_test, y_test = df_prep.to_train_test_samples('sample', 'Price')

Realizado en 1.0s


# 4. Modelamiento 

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

## 4.1 Ridge Regression

In [32]:
# Establecemos parámetros a evaluar en el modelo
ridge_grid = {'alpha': [0, .1, .2],
              'solver': ['sag', 'sparse_cg']}
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
ridge_reg = MLModel(model=Ridge(fit_intercept=True))
# Implementación del grid search
ridge_reg.grid_search(X_train,
                      y_train,
                      param_grid=ridge_grid,
                      n_jobs=-2,
                      cv=cv)

Mejores parámetros:
{'alpha': 0, 'solver': 'sag'}

Realizado en 119.0s


In [33]:
# Métricas
ridge_reg.metrics(X_test, y_test)

mse: 78978965.057
mae: 5408.43
r2: 0.574


{'mse': 78978965.057, 'mae': 5408.43, 'r2': 0.574}

In [32]:
# Instancibest_modelos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
linear2_reg = MLModel(model=LinearRegression(fit_intercept=True))
# Implementación del grid search
linear2_reg.fit(X_train, y_train)

Realizado en 1.0s


In [33]:
linear2_reg.metrics(X_test, y_test)

mse: 75390964.789
mae: 5440.391
r2: 0.581


{'mse': 75390964.789, 'mae': 5440.391, 'r2': 0.581}

## 4.2 LightGBM

In [34]:
# Establecemos parámetros a evaluar en el modelo
lgb_grid = {'max_depth': [3, 4, 5, 6], 
            'num_leaves': [30, 50, 60]}
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
lgb_reg = MLModel(model=LGBMRegressor(n_jobs=1,
                                      random_state=rd_seed))
# Implementación del grid search
lgb_reg.grid_search(X_train,
                    y_train,
                    param_grid=lgb_grid,
                    n_jobs=-2,
                    cv=cv)

Mejores parámetros:
{'max_depth': 6, 'num_leaves': 50}

Realizado en 18.0s


In [35]:
lgb_reg.metrics(X_test, y_test)

mse: 72340632.311
mae: 4913.508
r2: 0.609


{'mse': 72340632.311, 'mae': 4913.508, 'r2': 0.609}

## 4.3 XGBoost

In [41]:
# Establecemos parámetros a evaluar en el modelo
xgb_grid = {'max_depth': [3, 4, 5, 6], 
            'n_estimators': [50, 60, 70]}
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
xgb_reg = MLModel(model=XGBRegressor(objective ='reg:squarederror',
                                     n_jobs=1,
                                     seed=rd_seed))
# Implementación del grid search
xgb_reg.grid_search(X_train,
                    y_train,
                    param_grid=xgb_grid,
                    n_jobs=-2,
                    cv=cv)

Mejores parámetros:
{'max_depth': 6, 'n_estimators': 70}

Realizado en 589.0s


In [42]:
# Métricas
xgb_reg.metrics(X_test, y_test)

mse: 62266818.345
mae: 5056.726
r2: 0.654


{'mse': 62266818.345, 'mae': 5056.726, 'r2': 0.654}

## 4.4 RandomForest

In [ ]:
# Establecemos parámetros a evaluar en el modelo
rf_grid = {'n_estimators': [400, 500], 
           'max_depth': [3, 5, 7], 
           }
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
rf_reg = MLModel(model=RandomForestRegressor(oob_score=True,
                                             n_jobs=1,
                                             random_state=rd_seed))
# Implementación del grid search
rf_reg.grid_search(X_train,
                    y_train,
                    param_grid=rf_grid,
                    cv=cv)

In [ ]:
rf_reg.metrics(X_test, y_test)